In [14]:
import glob
import pickle
import numpy
from music21 import instrument, note, stream, chord, converter
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [15]:
# Run this once to create the notes file needed by generate()
# from music21 import converter is already imported above
import glob
import os

def extract_notes(input_glob='input/lofi-weights/*.mid', notes_path='input/lofi-weights/notes'):
    """Extract notes/chords from MIDI files and save to a pickle for training/inference."""
    notes = []
    midi_files = glob.glob(input_glob)
    print(f"Found {len(midi_files)} MIDI files")
    for fp in midi_files:
        try:
            midi = converter.parse(fp)
            parts = instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse() if parts else midi.flat.notes
            for el in notes_to_parse:
                if isinstance(el, note.Note):
                    notes.append(str(el.pitch))
                elif isinstance(el, chord.Chord):
                    notes.append('.'.join(str(n) for n in el.normalOrder))
        except Exception as exc:
            print(f"Skipping {os.path.basename(fp)}: {exc}")
    with open(notes_path, 'wb') as fh:
        pickle.dump(notes, fh)
    print(f"Saved {len(notes)} notes to {notes_path}")
    return notes

# Uncomment and run once to generate the notes file
extract_notes()


Found 93 MIDI files
Saved 1902 notes to input/lofi-weights/notes


['C#5',
 'B-4',
 'C#5',
 '10.1',
 '10.1',
 'B-4',
 'C#5',
 '10.1',
 'B-4',
 'C#5',
 '10.1',
 'B-4',
 'C#5',
 'B-4',
 'C#5',
 'B-4',
 'C#5',
 '10.1',
 'B-4',
 '10.1',
 'C#5',
 'B-4',
 'C#5',
 '10.1',
 'B-4',
 'E-5',
 'F5',
 'G#5',
 'B-5',
 'G#5',
 'F5',
 '1.3.5',
 'E-5',
 'C#5',
 '8.10',
 'B-4',
 'C#5',
 'E-5',
 'F5',
 'G#5',
 'B-5',
 'C#6',
 'E-6',
 'F6',
 'B-6',
 'G#6',
 'F6',
 '3.5',
 'E-6',
 'C#6',
 'B-5',
 'G#5',
 'F5',
 '3.5',
 'E-5',
 'C#5',
 '8.10',
 'F4',
 '2.5.7.10',
 '4.7.9.0',
 '9.10.2.5',
 '4.7.9.0',
 '2.5.7.10',
 '4.7.9.0',
 '10.2',
 '3.6.10',
 '3.5.10',
 '3.6.8.10.11',
 '10.1.3.5.6',
 '10.2.5',
 '3.6.10',
 '3.5.10',
 '3.6.8.10.11',
 '10.1.3.5.6',
 '10.2.5',
 '4.7.9.11.0',
 'E4',
 'G4',
 'A5',
 'E5',
 'A4',
 '4.5.9.0',
 'A4',
 '2.4.7.9',
 '4.7.9.0',
 'A4',
 '4.7.9.11.0',
 'E4',
 'G4',
 'A5',
 'E5',
 'A4',
 '4.5.9.0',
 'A4',
 '2.4.7.9',
 '4.7.9.0',
 'A4',
 'C#6',
 'E-6',
 'F6',
 'B-6',
 'G#6',
 'F6',
 'E-6',
 'F6',
 'C#6',
 'E-6',
 'F6',
 'B-6',
 'G#6',
 'F6',
 'E-6',
 'F6'

In [16]:


def generate():
    """Generate a piano midi file using trained weights and saved notes."""
    with open('input/lofi-weights/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    if not notes:
        raise ValueError("No notes found. Run extract_notes() to create input/lofi-weights/notes.")

    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(pitchnames)
    if n_vocab == 0:
        raise ValueError("Vocabulary is empty. Ensure extract_notes() parsed notes correctly.")

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)


def prepare_sequences(notes, pitchnames, n_vocab):
    """Prepare the sequences used by the Neural Network."""
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 32
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    if n_patterns == 0:
        raise ValueError("Not enough notes to build sequences. Need > sequence_length notes; rerun extract_notes().")

    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)


def create_network(network_input, n_vocab):
    """Create the structure of the neural network."""
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    weight_files = glob.glob('input/lofi-weights/*.hdf5')
    if not weight_files:
        raise FileNotFoundError(
            "No .hdf5 weights found in input/lofi-weights. "
            "Train a model or place weights in that folder."
        )
    weight_file = sorted(weight_files)[-1]
    print(f"Loading weights: {weight_file}")
    model.load_weights(weight_file)

    return model


def generate_notes(model, network_input, pitchnames, n_vocab):
    """Generate notes from the neural network based on a sequence of notes."""
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(200):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


def create_midi(prediction_output):
    """Convert the output from the prediction to notes and create a midi file."""
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='lofi_output.mid')


if __name__ == '__main__':
    generate()


FileNotFoundError: No .hdf5 weights found in input/lofi-weights. Train a model or place weights in that folder.

In [7]:
#after converting to mp3...
import IPython
IPython.display.Audio("/input/lofi-weights/lofi_test_output2.mp3")

ValueError: rate must be specified when data is a numpy array or list of audio samples.